# Setup

In [ ]:
import random
import re
import pprint

import matplotlib.pyplot as plt

from helium import helium, ops, graphs
from helium.runtime import HeliumServerConfig

# Use local Helium server. You cannot use remote Helium server for this example.
server_config = HeliumServerConfig(
    is_local=True, llm_service_configs="../configs/llm_services.json"
)

In [ ]:
# Start the Helium server locally
helium.get_started_instance(config=server_config)

# Guessing game

In [ ]:
def check_resp(inputs: tuple[ops.SingleDtype, ...]) -> str:
    s, answer = inputs
    assert isinstance(s, str) and isinstance(answer, str)
    guess = re.search(r"(\d+)", s)
    if guess and int(answer) < int(guess.group()):
        return "high"
    return "low"


def build_guessing_game_graph(
    num_agents: int, num_iter: int
) -> tuple[list[int], graphs.Graph]:
    answers = random.choices(range(1, 101), k=num_agents)
    answer_data = ops.data([str(i) for i in answers])
    chat_instr = ops.message_data(
        [
            ops.OpMessage(role="system", content="You are a helpful assistant."),
            ops.OpMessage(
                role="user", content=["Guess a number between 1 and 100"] * num_agents
            ),
        ]
    )
    llm_chat = ops.llm_chat(chat_instr, return_history=True)
    llm_resp = ops.get_last_message(llm_chat)
    check = ops.lambda_op([llm_resp, answer_data], check_resp)
    new_msg = ops.format_op("Too {check}. Try again.", check=check)
    new_instr = ops.append_message(llm_chat, new_msg)

    cmpl_prompt = ops.data("My name is")
    llm_cmpl = ops.llm_completion(cmpl_prompt, echo=True)

    chat, _, cmpl = ops.loop(
        [None, chat_instr, cmpl_prompt],
        [llm_chat, new_instr, llm_cmpl],
        num_iter=num_iter,
    )

    chat_out = ops.as_output("chat", chat)
    cmpl_out = ops.as_output("cmpl", cmpl)

    return answers, graphs.from_ops([chat_out, cmpl_out])

In [ ]:
# Build the writing simulation graph
answers, graph = build_guessing_game_graph(num_agents=5, num_iter=3)

In [ ]:
# Visualize the graph
plt.figure(figsize=(12, 12))
_ = graph.visualize(layout="planar")

In [ ]:
# Execute the graph on the Helium server
result = helium.invoke(graph)

In [ ]:
# Print the results
print("Answers:", answers)
pprint.pprint(result)